# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

12


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
#Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS tiago 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('tiago')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4  

table_name = 'session_details'
############################## CREATE ##############################
# Delete table if already exists
delete_table_query = f"DROP TABLE IF EXISTS {table_name}"
try:
    session.execute(delete_table_query)
except Exception as e:
    print(e)
    
# Create empty Table with columns:
#    (name of the artist)     : artist
#    (title of the song)      : song_title
#    (item number in session) : item_in_session
#    (lenght of the song)     : song_length
# Primary keys necessary to have unique rows and answer the query:
# session_id
# item_in_session   
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} 
    (artist_name text, song_title text, item_in_session int, song_length float, session_id int, 
    PRIMARY KEY (session_id, item_in_session))
"""
try:
    session.execute(create_table_query)
except Exception as e:
    print(e)
    
############################## INSERT ############################## 
# Insert rows into Table
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = f"INSERT INTO {table_name} (artist_name, song_title, item_in_session, song_length, session_id)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], int(line[3]), float(line[5]), int(line[8])))
        
############################## QUERY ##############################   
# Query Table 
query1 = f"SELECT artist_name, song_title, song_length from {table_name} WHERE session_id = 338 and item_in_session = 4"
rows = session.execute(query1)
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

In [9]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182

table_name = 'artist_and_song_by_users'
############################## CREATE ##############################
# Delete table if already exists
delete_table_query = f"DROP TABLE IF EXISTS {table_name}"
try:
    session.execute(delete_table_query)
except Exception as e:
    print(e)

# Create empty Table with columns:
#    (name of the artist)     : artist
#    (title of the song)      : song_title
#    (item number in session) : item_in_session
#    (first name of the user) : f_name
#    (last name of the user)  : l_name
#    (unique id of each user) : user_id
#    (id of the session)      : session_id
# Primary keys necessary to have unique rows and answer the query:
# session_id
# user_id  
# item_in_session
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} 
    (artist text, song_title text, item_in_session int, f_name text, l_name text, user_id int, session_id int, 
    PRIMARY KEY ((user_id, session_id), item_in_session))
"""
try:
    session.execute(create_table_query)
except Exception as e:
    print(e)

############################## INSERT ##############################   
# Insert rows into Table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = f"INSERT INTO {table_name} (artist, song_title, item_in_session, f_name, l_name, user_id, session_id)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], int(line[3]), line[1], line[4], int(line[10]), int(line[8])))

############################## QUERY ############################## 
# Query Table
query1 = f"SELECT artist, song_title, f_name, l_name from {table_name} \
            WHERE user_id = 10 and session_id = 182 ORDER BY item_in_session"
rows = session.execute(query1)
for row in rows:
    print (row.artist, row.song_title, row.f_name, row.l_name)
                    

In [10]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

table_name = 'users_by_song'
############################## CREATE ##############################
# Delete table if already exists
delete_table_query = f"DROP TABLE IF EXISTS {table_name}"
try:
    session.execute(delete_table_query)
except Exception as e:
    print(e)

# Create empty Table with columns:
#    (title of the song)      : song_title
#    (first name of the user) : f_name
#    (last name of the user)  : l_name
#    (unique id of each user) : user_id
# Primary keys necessary to have unique rows and answer the query:
# song_title
# user_id
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} 
    (song_title text, f_name text, l_name text, user_id int, PRIMARY KEY (song_title, user_id))
"""
try:
    session.execute(create_table_query)
except Exception as e:
    print(e)

############################## INSERT ##############################   
# Insert rows into Table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = f"INSERT INTO {table_name} (song_title, f_name, l_name, user_id)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

############################## QUERY ############################## 
# Query Table
query1 = f"SELECT f_name, l_name from {table_name} WHERE song_title = 'All Hands Against His Own'"
rows = session.execute(query1)
for row in rows:
    print (row.f_name, row.l_name)

### Drop the tables before closing out the sessions

In [11]:
# Drop the table before closing out the sessions
delete_table_query = "DROP TABLE IF EXISTS session_details"
session.execute(delete_table_query)

delete_table_query = "DROP TABLE IF EXISTS artist_and_song_by_users"
session.execute(delete_table_query)

delete_table_query = "DROP TABLE IF EXISTS users_by_song"
session.execute(delete_table_query)

### Close the session and cluster connection¶

In [12]:
session.shutdown()
cluster.shutdown()